<p style="padding:5px;background-color: #007f99 ;border:1px solid black;border-radius:20px;font-weight:bold;font-size:2em;padding:15px;text-align:center;"> 
   Arbres et formules propositionelles
    <br>

</p>

Ce TP a pour but de passer en revue l'ensemble du programme d'option info en MPSI, et de réviser les éléments du language OCaml correspondants.

!!! danger Avant toute chose
Attention, quelques consignes sur le code ! 
- Les noms des variables doivent être explicites, en particulier lorsque leur rôle n'est pas évident. On tolère `l` et sa décomposition `x::q` pour une liste, `f` pour une formule, `i` pour un indice... Mais restez cohérents.
- Votre code doit être commenté, et les types des arguments ainsi que le type de retour des fonctions doit être précisé. C'est d'autant plus vrai pour les fonctions auxiliaires.
- Vous devez testez chacune des fonction que vous écrivez. Vous disposez d'une variable `formule_test` à cet effet. 
!!!

# Formules propositionelles et notation préfixe
## 1. Définitions
Une **formule propositionelle** est définie inductivement comme 
- soit une variable $x$
- soit, si $e$ est une formule, $\neg e$ qui signifie "l'inverse de e"
- soit, si $e_1$ et $e_2$ sont des formules, 
    - $e_1 \land e_2$ pour le "et"
    - $e_1 \lor e_2$ pour le "ou"
    
    
Par exemple, $(a \lor b) \land (\neg a \lor \neg b)$ est une formule.

La valeur de vérité d'une formule, pour une valuation de ses variables (c'est à dire qu'on assigne à chaque variable "vrai" ou "faux"), est le résultat de l'opération logique usuelle sur la formule, dans laquelle on a remplacé chaque variable par sa valeur.

!!! question Question 1
Soit les variables $a, b, c$. Écrivez des formules qui sont vraies seulement dans les cas suivants :
- Si les trois variables sont vraies
- Si exactement une des trois variable est vraie
- Si les trois variables n'ont pas la même valeur
!!!

Une formule propositionelle sous **notation préfixe** est une formule propositionelle où, dans le cas des opérateurs binaires, on écrit $\land e_1 e_2$ au lieu de $e_1 \land e_2$ (et de même pour $\lor$).

!!! tip Avantage
Avec la notation préfixe, il n'est plus nécéssaire d'utiliser des parenthèses ! Essayez de comprendre pourquoi.
!!!

!!! question Question 2
Transformez les formules suivantes en notation préfixe, en des formules en notation usuelle :
- $\land a \; b$
- $\land \lor a \; b \; c$
- $\lor \neg \land a \; \neg b \land \neg a \; b$
!!!

## 2. Formules en OCaml

On va dans un premier temps représenter en OCaml les formules en notation préfixe. Pour cela, on utilise un type de liste

In [ ]:
type formule = char list;;

Dans une formule, le caractère `*` représente le $\land$, `+` représente le $\lor$, et `!` représente le $\neg$. Tous les autres symboles représentent des variables.

!!! example
Par exemple, la formule $\lor \neg \land a \; \neg b \land \neg a \; b$ en notation préfixe, sera représentée par la liste `['+'; '!'; '*'; 'a'; '!'; 'b'; '*'; '!'; 'a'; 'b']`
!!!

In [ ]:
let formule_test = ['+'; '!'; '*'; 'a'; '!'; 'b'; '*'; '!'; 'a'; 'b'];;

!!! danger Rappel
Si vous n'avez pas lu l'encadré en haut du TP, c'est le moment.
!!!

!!! question Question 3
Compléter les fonctions suivantes, qui appliquent des opérations à des formules. La négation est déjà complétée à titre d'exemple.
!!!

!!! info Rappels de OCaml : fonctions et variables
- En OCaml, les fonctions se déclarent avec 
```ocaml
let nom_de_la_fonction (arg1: type1, arg2: type2): type_de_retour = 
    (*corps de la fonction*)
```
- À l'interieur d'une fonction, les variables se déclarent avec `let [nom] = [valeur] in`
- Si `l1` et `l2` sont deux listes, `l1 @ l2` représente leur concaténation, et `x::l1` représente la liste `l1` à laquelle on a ajouté la valeur `x` au début.
!!!

In [ ]:
(*Renvoie la négation d'une formule*)
let neg (f: formule): formule = '!'::f

In [ ]:
(*Renvoie une formule consituée d'une seule variable*)
let var (v: char): formule = (*à compléter*)

In [ ]:
(*Renvoie la conjonction ("et") de deux formules*)
let et (f1: formule) (f2: formule): formule = (*à compléter*)

In [ ]:
(*Renvoie la disjonction ("ou") de deux formules*)
let ou (f1: formule) (f2: formule): formule = (*à compléter*)

!!! question Question 4
À l'aide des fonctions ci-dessus (et uniquement elles), réécrivez la formule `formule_test`.
!!!

!!! info Rappels de OCaml : fonctions récursives
En OCaml, on peut écrire des fonctions *récursives* avec le mot-clé `let rec`. De plus, on peut faire du *pattern matching* sur les listes, avec `match .. with ..`.

En combinant les deux, on obtient une construction **très** commune : 
```OCaml
let rec f (l: 'a list) = match l with 
    |[] -> (*cas de base, lorsque la liste est vide*)
    |x::q -> (*cas récursif : faire quelque chose avec x, et appeler (f q).*)
```

!!!

!!! info Rappels de OCaml : if
Pour écrire une condition, on utilise une construction `if`. 
```OCaml
if (condition) then [valeur 1] 
else [valeur 2]
```
À noter que le `if` en OCaml a une valeur ! Par exemple, on peut écrire
```OCaml
let verifie_couleur = if couleur = bleu then true else false
```

!!!

!!! question Question 5
Écrivez une fonction récursive `nombre_variables` de type `formule -> int`, qui prend en argument une formule, et renvoie le nombre de variables dans la formule (_sans_ enlever les doublons).

Vous pourrez vous aider d'une fonction auxilliaire `est_variable` qui renvoie `true` si le `char` donné en argument est une variable, et `false` sinon.

Donnez sa complexité en temps et en espace.
!!!

## 3. Formules sous forme d'arbres
On va maintenant étudier une autre représentation des formules, plus proche de la forme "habituelle".

!!! info Rappels de OCaml : types énumération
En OCaml, il est possible de déclarer des types "énumération", de la manière suivante : 
```OCaml
type classe = MP | MPSI | PCSI | PC
```
Un élément de ce type a 4 valeurs possible, une pour chaque variant défini. 

On peut aussi ajouter des données à un variant avec le mot-clé `of`. Par exemple : 
```OCaml
type classe = MP | MPSI | PCSI | PC | Autre of string
```

Enfin, il est possible stocker une donnée du type lui-même dans un variant ! Par exemple, si on veut redéfinir le type `list` pour les entiers, on peut écrire
```OCaml
type liste_entier = Vide | Element of (int * liste_entier)
```

`int * liste_entier` est ici le type d'un tuple de deux éléments, un entier et une liste d'entiers.
!!!

!!! question Question 6
On a commencé à définir un type `arbre_formule`. Complétez la définition pour ajouter les variants manquants. 
!!!

In [ ]:
type arbre_formule = 
     Var of char
    |Neg of arbre_formule
    (*à compléter*)
    ;;

!!! question Question 7
Définissez une variable `arbre_formule_test` qui contient la version _arbre_ de `formule_test`.
!!!

!!! info Rappels de OCaml : *pattern-matching* sur des énumérations
On peut également faire du *pattern-matching* sur les énumérations ! Par exemple, avec le type classe de l'exemple précédent, cela donnerait : 
```OCaml
let nombre_heure_math (c: classe): int = match c with 
    |MP -> 10
    |MPSI -> 8
    |PCSI -> 7
    |PC -> 7
    |Autre (s) -> (print_string "Classe inconnue : "; print_string s; -1)
```

Je reviendrais sur l'impression et l'utilisation des `;` dans la bulle suivante.
!!!

!!! question Question 8
Écrivez une fonction `nombre_variable_arbre` de type `arbre_formule -> int` qui renvoie le nombre de variables dans un arbre de formule. Vérifiez que votre fonction renvoie bien la même chose sur `arbre_formule_test` que `nombre_variable` sur `formule_test`.
!!!

!!! info Rappels de OCaml : instructions et impression
- Pour afficher une valeur en OCaml, il faut utiliser la fonction qui correspond au type de l'objet. Les fonctions disponibles sont : `print_int`, `print_char`, `print_string`, `print_float`. Pour revenir à la ligne, on utilise `print_newline ()` (attention à bien ajouter () à la fin !).
- En OCaml, certaines fonctions renvoie un type particulier, le type `unit`. En particulier, c'est le cas de toutes les fonctions d'affichage. Il devient alors possible de "chainer" ces fonctions, c'est à dire d'en utiliser plusieurs à la suite. Pour cela, on utilise `;` pour les séparer.
- Lorsqu'on utilise des `;`, ou bien plusieurs `if` ou `match`, il est bon de mettre des parenthèses pour délimiter les blocs. 

Je vous fournis une fonction `print_char_list` comme exemple. Notez qu'elle renvoie le type `unit`, c'est à dire qu'elle ne renvoie rien.
!!!

In [ ]:
(*affiche une liste de caractères*)
let print_char_list (l: char list): unit = 
    (*Cette fonction auxiliaire effectue l'affichage en récursif, excépté les crochets*)
    let rec aux (l: char list): unit =
        match l with 
            |[] -> ()
            |x::q -> (print_char x; print_string ", "; aux q)
    in
    print_char '['; aux l; print_char ']'; print_newline ()

In [ ]:
print_char_list formule_test

!!! question Question 9
Écrivez une fonction de type `arbre_formule -> unit` qui permet d'afficher un arbre de formule, et testez-là sur `formule_test`. 
!!!
!!! tip Indice
Vous avez plusieurs options : 
- afficher l'arbre comme une formule dans la notation usuelle (donc avec les opérateurs entre leurs opérandes)
- (plus compliqué) afficher l'arbre sous forme d'arbre. Je vous conseille alors de calculer au passage la profondeur de chaque noeud, et de l'afficher avec une identation correspondant à sa profondeur. 
!!!

!!! question Question 10
Écrivez une fonction récursive de type `arbre_formule -> formule` qui prend en argument une formule sous forme d'arbre, et renvoie la même formule en notation préfixe.

Donnez sa complexité en temps et en espace.
!!!

## 4. Evaluation d'une formule

On va procéder à l'évalutation de formules en forme arborescentes. Pour cela, on va utiliser une table de hashage (`Hashtbl`), qui est une structure associative qui implémente une interface de dictionnaire. L'utilisation d'une `Hashtbl` n'est pas exigible sans rappel, mais on peut vous la demander en rappelant son interface. C'est donc ce que vous allez faire ici. 

!!! info Interface de `Hashtbl` en OCaml 
- Création d'une table : `Hashtbl.create n` créé une table avec une capacité initale `n`, renvoie `unit`
- Ajout d'un élément : `Hashtbl.add table clé valeur`, renvoie `unit`
- Récupération d'un élément : `Hashtbl.find table clé`, renvoie la valeur associée à la clé (ou une erreur si elle n'existe pas)
- Autre option : `Hashtbl.find_opt table clé`, renvoie `Some (valeur)` si la valeur existe, et `None` sinon.
- Test d'existence : `Hashtbl.mem table clé`, renvoie `true` si la clé est présente, `false` sinon.
- Supression : `Hashtbl.remove table clé`, renvoie `unit`.
- Parcours : `Hashtbl.iter fonction table` applique une fonction de type `clé -> valeur -> unit` à chaque élément de la table.

!!!

!!! question Question 11
Créer une table de hashage, puis ajoutez-y les correspondances suivantes : 
| clé | valeur  |
|---|---------|
| 'a' | `true`  |
| 'b' | `true`  |
| 'c' | `false` |
| 'd' | `false` |

Quel est le type de votre table, après ajout des valeurs ? 
!!!

!!! question Question 12
Ecrivez une fonction récursive `eval_arbre` de type `arbre_formule -> (char, bool) Hashtbl.t -> bool` qui renvoie la valeur d'une formule si toutes les variables de la formule sont fixées dans la table, et renvoie une erreur sinon.

Donnez sa complexité en temps et en espace.
!!!

!!! info Rappels de OCaml : les erreurs
Il y a plusieurs façon de gérer les erreurs en OCaml, mais la manière la plus simple est d'utiliser l'instruction `failwith "raison de l'erreur"`. Elle peut correspondre à tous les types, pour plus de facilité d'utilisation.
!!!

!!! question Question 13
Écrivez une fonction `eval_formule` de type `formule -> (char, bool) Hashtbl.t -> bool` qui renvoie la valeur d'une formule si toutes les variables de la formule sont fixées dans la table, et une erreur sinon.

Donnez sa complexité en temps et en espace.
!!!

!!! hint Indice
Vous pouvez utiliser une fonction auxiliaire récursive qui calcule la valeur de la première formule valable d'une liste, et renvoie cette valeur ainsi que la liste où on a supprimé les caractère correspondants.

Par exemple, pour la liste `["a"; "+"; "b"; "c"]`, la fonction renverra `(true, ["+"; "b"; "c"])`. Sur `["+"; "b"; "c"]`, elle renverra la valeur de $b \lor c$ (après avoir faire deux appels récursifs pour les arguments du `+`).
!!!

!!! question Question 14
Écrivez une fonction de type `formule -> arbre_formule` qui convertit une formule en notation préfixe, en une formule sous forme d'arbre.

Donnez sa complexité en temps et en espace.
!!!

!!! hint Indice
C'est presque la même fonction qu'à la question précédente !
!!!

!!! question Question 15
Commentez sur les complexités des différentes fonction pour évaluer les formules et sur les différences entre les structures de données
!!!